In [1]:
from cot import Collection

In [2]:
coll = Collection("open_book_qa", load_pregenerated_cots=False)
coll

Reusing dataset open_book_qa_dataset (/home/kon/.cache/huggingface/datasets/open_book_qa_dataset/thoughtsource/1.0.0/b1163057b19c0b0be4f0f1104103cb49d35de01177baec5250adb5246116272f)


Loading open_book_qa...


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4957 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

| Name         |   Train |   Valid |   Test |
|--------------|---------|---------|--------|
| open_book_qa |    4957 |     500 |    500 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'med_qa_open', 'medmc_qa', '_init_', 'mmlu_clinical_knowledge', 'mmlu_college_biology', 'mmlu_college_medicine', 'mmlu_medical_genetics', 'mmlu_professional_medicine', '_init_', 'mmlu_anatomy', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [3]:
coll = coll.select(split = "train", number_samples=1)
coll

| Name         |   Train | Valid   | Test   |
|--------------|---------|---------|--------|
| open_book_qa |       1 | -       | -      |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'med_qa_open', 'medmc_qa', '_init_', 'mmlu_clinical_knowledge', 'mmlu_college_biology', 'mmlu_college_medicine', 'mmlu_medical_genetics', 'mmlu_professional_medicine', '_init_', 'mmlu_anatomy', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [4]:
# coll['open_book_qa']['train'][0]

In [ ]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
prin